# Notebook pour la baseline de classification des affiches de cinéma

**Ce Notebook sert de baseline pour la classification d'affiches de cinéma, pour le colloque Humanistica (Genève, 2022)**

*Plan du Notebook*

> Importer les données

> Charger et appliquer le modèle

> Enregistrer les métriques

In [ ]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from sklearn.metrics import f1_score
from tensorflow.keras.metrics import Precision, Recall
%pip install tensorflow_addons
import tensorflow_addons as tfa
from tensorflow_addons.metrics import F1Score

## Importer les données

In [ ]:
# Dimensions des images d'entrée
image_width, image_height = 128, 128

# Chemins d'accès vers les données d'entraînement, de validation et de test
train_data_dir = ""
validation_data_dir = ""
test_data_dir = ""

In [ ]:
# Prétraitement des images
train_data_generator = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
validation_data_generator = ImageDataGenerator(rescale=1./255)
test_data_generator = ImageDataGenerator(rescale=1./255)

# Chargement des données d'entraînement
train_generator = train_data_generator.flow_from_directory(train_data_dir, target_size=(image_width, image_height), batch_size=32, class_mode='categorical', shuffle=True, seed=42)
# Chargement des données de validation
validation_generator = validation_data_generator.flow_from_directory(validation_data_dir, target_size=(image_width, image_height), batch_size=32, class_mode='categorical', shuffle=True, seed=42)
# Chargement des données de test
test_generator = test_data_generator.flow_from_directory(test_data_dir, target_size=(image_width, image_height), batch_size=32, class_mode='categorical', shuffle=True, seed=42)

## Charger et appliquer le modèle

In [ ]:
# Création du modèle CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_width, image_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [ ]:
# Compilation du modèle
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', Precision(), Recall(), F1Score(num_classes=3)])

In [ ]:
# Entraînement du modèle
epochs = 10
model.fit_generator(train_generator, steps_per_epoch=len(train_generator), epochs=epochs, validation_data=validation_generator, validation_steps=len(validation_generator))

## Enregistrer les métriques

In [ ]:
# Évaluation du modèle sur les données de test
results = model.evaluate(test_generator, steps=len(test_generator))
print("test loss, test acc:", results)

In [ ]:
# enregistrer

# Création du dictionnaire
data = {
    'model': nom_du_model,
    'test_loss': results[0],
    'test_accuracy': results[1],
    'test_precision': results[2],
    'test_recall': results[3],
    'test_f1_score': results[4],
}

# Conversion du dictionnaire en DataFrame
df = pd.DataFrame(data)

# Enregistrement du DataFrame dans un fichier CSV
df.to_csv(f'test_mesures_{nom_du_model}.csv')